## Absenteeism project

## Import the relevant libraries

In [1]:
import pandas as pd
import numpy as np

## Load the data

In [2]:
data_preprocessed = pd.read_csv('1.1 Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pet,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


## Create the targets

In [4]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] >
                data_preprocessed['Absenteeism Time in Hours'].median() ,1,0)

In [6]:
data_preprocessed['Excessive Absenteeism'] = targets

In [7]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pet,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week', 'Daily Work Load Average','Distance to Work'],axis=1)

## Select inputs

In [9]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

## Standardize the inputs

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [11]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pet'], dtype=object)

In [12]:
# choose the columns to scale
# we later augmented this code and put it in comments
# columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work',
       #'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pet']
    
# select the columns to omit
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [13]:
# create the columns to scale, based on the columns to omit
# use list comprehension to iterate over the list
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [14]:
# declare a scaler object, specifying the columns you want to scale
absenteeism_scaler = CustomScaler(columns_to_scale)

In [15]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pet'],
             copy=None, with_mean=None, with_std=None)

In [16]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [17]:
scaled_inputs.shape

(700, 11)

## Split the data into train, test and split

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state=20)

## Logistic Regression with sklearn

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 

## Training the model

In [21]:
reg = LogisticRegression()

In [22]:
reg.fit(x_train,y_train)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
reg.score(x_train,y_train)

0.775

## Manually check the accuracy

In [24]:
model_outputs = reg.predict(x_train)

In [25]:
np.mean((model_outputs == y_train))

0.775

## Find the coeficients and the intercept

In [26]:
feature_name = unscaled_inputs.columns.values

In [27]:
summary = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary['Coeficients'] = np.transpose(reg.coef_)
summary.index = summary.index +1
summary.loc[0] = ['Intercept',reg.intercept_[0]]
summary = summary.sort_index()
summary

,Feature name,Coeficients
0,Intercept,-1.431381
1,Reason_1,2.602372
2,Reason_2,0.843500
3,Reason_3,2.940787
4,Reason_4,0.637234
5,Month Value,0.005651
6,Transportation Expense,0.619534
7,Age,-0.176355
8,Body Mass Index,0.284103
9,Education,-0.263725


## Interpreting the coeficients

In [28]:
summary['Odds_ratio'] = np.exp(summary.Coeficients)
summary

,Feature name,Coeficients,Odds_ratio
0,Intercept,-1.431381,0.238979
1,Reason_1,2.602372,13.495716
2,Reason_2,0.843500,2.324489
3,Reason_3,2.940787,18.930743
4,Reason_4,0.637234,1.891243
5,Month Value,0.005651,1.005667
6,Transportation Expense,0.619534,1.858062
7,Age,-0.176355,0.838320
8,Body Mass Index,0.284103,1.328570
9,Education,-0.263725,0.768185


In [29]:
summary.sort_values('Odds_ratio',ascending=False)

,Feature name,Coeficients,Odds_ratio
3,Reason_3,2.940787,18.930743
1,Reason_1,2.602372,13.495716
2,Reason_2,0.843500,2.324489
4,Reason_4,0.637234,1.891243
6,Transportation Expense,0.619534,1.858062
10,Children,0.351950,1.421838
8,Body Mass Index,0.284103,1.328570
5,Month Value,0.005651,1.005667
7,Age,-0.176355,0.838320
9,Education,-0.263725,0.768185


## Testing the model

In [30]:
reg.score(x_test,y_test)

0.7357142857142858

In [31]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.75308922, 0.24691078],
       [0.60926091, 0.39073909],
       [0.4859575 , 0.5140425 ],
       [0.7552847 , 0.2447153 ],
       [0.0839675 , 0.9160325 ],
       [0.30192695, 0.69807305],
       [0.30166774, 0.69833226],
       [0.1151045 , 0.8848955 ],
       [0.73775967, 0.26224033],
       [0.75403176, 0.24596824],
       [0.50719215, 0.49280785],
       [0.19719276, 0.80280724],
       [0.06163196, 0.93836804],
       [0.70917025, 0.29082975],
       [0.29280547, 0.70719453],
       [0.5241047 , 0.4758953 ],
       [0.50676929, 0.49323071],
       [0.50888352, 0.49111648],
       [0.367008  , 0.632992  ],
       [0.06355661, 0.93644339],
       [0.73644831, 0.26355169],
       [0.7552847 , 0.2447153 ],
       [0.47457156, 0.52542844],
       [0.47288443, 0.52711557],
       [0.22026535, 0.77973465],
       [0.73808685, 0.26191315],
       [0.51184512, 0.48815488],
       [0.87683579, 0.12316421],
       [0.23445563, 0.76554437],
       [0.7552847 , 0.2447153 ],
       [0.

## Save the model

In [32]:
import pickle

In [33]:
with open('model','wb') as file:
    pickle.dump(reg,file)

In [34]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler,file)